<a href="https://colab.research.google.com/github/Research-SLIIT/Financial-Risk-Analyisis-Model/blob/main/antigravity_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# ============================================================================
# 1. IMPORTS AND SETUP
# ============================================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import mutual_info_regression
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("BALANCED Z-SCORE PREDICTION MODEL - HIGH ACCURACY")
print("Optimized for Strong Cross-Validation Performance")
print("=" * 80)

BALANCED Z-SCORE PREDICTION MODEL - HIGH ACCURACY
Optimized for Strong Cross-Validation Performance


In [15]:
# ============================================================================
# 2. LOAD AND PREPROCESS DATA
# ============================================================================
print("\n" + "-" * 80)
print("DATA LOADING & PREPROCESSING")
print("-" * 80)

file_path = '/content/Dataset.csv'  # Update path if needed
df = pd.read_csv(file_path)

print(f"\n✓ Dataset loaded: {df.shape}")

df['QuarterDate'] = pd.to_datetime(df['QuarterDate'])
df = df.sort_values(['Company', 'QuarterDate']).reset_index(drop=True)

base_features = [
    'working_capital_to_total_assets',
    'retained_earnings_to_total_assets',
    'ebit_to_total_assets',
    'mve_to_total_liabilities',
    'sales_to_total_assets',
    'current_ratio',
    'debt_to_equity_ratio',
    'net_profit_margin',
    'z_score'
]

target_column = 'z_score_next_quarter'
df_clean = df.dropna(subset=[target_column]).copy()

# Fill missing values
for col in base_features:
    if df_clean[col].isnull().any():
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

print(f"✓ Preprocessing complete: {len(df_clean)} samples")


--------------------------------------------------------------------------------
DATA LOADING & PREPROCESSING
--------------------------------------------------------------------------------

✓ Dataset loaded: (586, 12)
✓ Preprocessing complete: 586 samples


In [16]:
# ============================================================================
# 3. BALANCED FEATURE ENGINEERING
# ============================================================================
print("\n" + "-" * 80)
print("BALANCED FEATURE ENGINEERING")
print("-" * 80)

df_features = df_clean.copy()

# Core composite indicators
df_features['financial_health_score'] = (
    df_features['current_ratio'] * 0.3 +
    df_features['z_score'] * 0.4 +
    (1 / (df_features['debt_to_equity_ratio'] + 0.1)) * 0.3
)

df_features['profitability_composite'] = (
    df_features['net_profit_margin'] * 0.4 +
    df_features['ebit_to_total_assets'] * 0.3 +
    df_features['retained_earnings_to_total_assets'] * 0.3
)

df_features['operational_efficiency'] = (
    df_features['sales_to_total_assets'] /
    (df_features['working_capital_to_total_assets'].abs() + 0.01)
)

df_features['leverage_risk'] = (
    df_features['debt_to_equity_ratio'] / (df_features['current_ratio'] + 0.1)
)

# IMPORTANT: Keep z_score polynomial features (they were top performers)
df_features['z_score_squared'] = df_features['z_score'] ** 2
df_features['net_profit_margin_squared'] = df_features['net_profit_margin'] ** 2

# Market value interactions
df_features['mve_profitability'] = (
    df_features['mve_to_total_liabilities'] * df_features['net_profit_margin']
)

# Time-series features
df_features = df_features.sort_values(['Company', 'QuarterDate'])

# Lag features (1, 2, 3 quarters)
for col in ['z_score', 'net_profit_margin', 'current_ratio', 'sales_to_total_assets']:
    df_features[f'{col}_lag1'] = df_features.groupby('Company')[col].shift(1)
    df_features[f'{col}_lag2'] = df_features.groupby('Company')[col].shift(2)
    df_features[f'{col}_lag3'] = df_features.groupby('Company')[col].shift(3)

# Rolling statistics (3 quarters only)
for col in ['z_score', 'net_profit_margin', 'sales_to_total_assets']:
    df_features[f'{col}_ma3'] = (
        df_features.groupby('Company')[col]
        .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
    )
    df_features[f'{col}_std3'] = (
        df_features.groupby('Company')[col]
        .transform(lambda x: x.rolling(window=3, min_periods=1).std())
    )

# Momentum features
for col in ['z_score', 'net_profit_margin']:
    df_features[f'{col}_change'] = df_features.groupby('Company')[col].diff()
    df_features[f'{col}_pct_change'] = df_features.groupby('Company')[col].pct_change()

# Fill NaN values
df_features = df_features.fillna(method='bfill').fillna(method='ffill').fillna(0)

# Get all engineered features
engineered_features = [col for col in df_features.columns
                       if col not in ['Company', 'QuarterDate', target_column]]

# Remove infinite values
X_full = df_features[engineered_features].copy()
X_full = X_full.replace([np.inf, -np.inf], np.nan)
X_full = X_full.fillna(X_full.median())

y_full = df_features[target_column].copy()

print(f"✓ Created {len(engineered_features)} features")
print(f"  (Base: {len(base_features)}, Engineered: {len(engineered_features) - len(base_features)})")


--------------------------------------------------------------------------------
BALANCED FEATURE ENGINEERING
--------------------------------------------------------------------------------
✓ Created 38 features
  (Base: 9, Engineered: 29)


In [17]:
# ============================================================================
# 4. MODERATE FEATURE SELECTION (Keep 30-35 features)
# ============================================================================
print("\n" + "-" * 80)
print("MODERATE FEATURE SELECTION (TARGET: 30-35 FEATURES)")
print("-" * 80)

# Calculate mutual information scores
mi_scores = mutual_info_regression(X_full, y_full, random_state=42)
mi_scores_df = pd.DataFrame({
    'Feature': engineered_features,
    'MI_Score': mi_scores
}).sort_values('MI_Score', ascending=False)

# Select top 35 features (or top 60% if dataset is smaller)
n_features_to_keep = min(35, int(len(engineered_features) * 0.60))
selected_features = mi_scores_df.head(n_features_to_keep)['Feature'].tolist()

print(f"✓ Feature selection complete")
print(f"  Total features: {len(engineered_features)}")
print(f"  Selected features: {len(selected_features)}")
print(f"  Reduction: {100*(1-len(selected_features)/len(engineered_features)):.1f}%")

print(f"\n  Top 15 features by importance:")
for idx, row in mi_scores_df.head(15).iterrows():
    print(f"    {list(mi_scores_df.head(15).index).index(idx)+1}. {row['Feature']}: {row['MI_Score']:.4f}")

X_selected = X_full[selected_features].copy()


--------------------------------------------------------------------------------
MODERATE FEATURE SELECTION (TARGET: 30-35 FEATURES)
--------------------------------------------------------------------------------
✓ Feature selection complete
  Total features: 38
  Selected features: 22
  Reduction: 42.1%

  Top 15 features by importance:
    1. z_score: 0.7875
    2. z_score_squared: 0.7736
    3. z_score_ma3: 0.7324
    4. financial_health_score: 0.6768
    5. z_score_lag3: 0.6000
    6. z_score_lag1: 0.5889
    7. z_score_lag2: 0.5198
    8. mve_to_total_liabilities: 0.4630
    9. leverage_risk: 0.4593
    10. current_ratio: 0.4561
    11. working_capital_to_total_assets: 0.3963
    12. current_ratio_lag3: 0.3895
    13. current_ratio_lag2: 0.3852
    14. current_ratio_lag1: 0.3839
    15. debt_to_equity_ratio: 0.3808


In [18]:
# ============================================================================
# 5. TIME-SERIES TRAIN/TEST SPLIT
# ============================================================================
print("\n" + "-" * 80)
print("TIME-SERIES TRAIN/TEST SPLIT")
print("-" * 80)

dates = df_features['QuarterDate']
split_date = dates.quantile(0.80)

train_mask = dates <= split_date
test_mask = dates > split_date

X_train = X_selected[train_mask]
X_test = X_selected[test_mask]
y_train = y_full[train_mask]
y_test = y_full[test_mask]

train_dates = dates[train_mask]
test_dates = dates[test_mask]

print(f"✓ Split based on date: {split_date.date()}")
print(f"  Train: {len(X_train)} samples ({train_dates.min().date()} to {train_dates.max().date()})")
print(f"  Test:  {len(X_test)} samples ({test_dates.min().date()} to {test_dates.max().date()})")

# Feature scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ Features scaled (RobustScaler)")


--------------------------------------------------------------------------------
TIME-SERIES TRAIN/TEST SPLIT
--------------------------------------------------------------------------------
✓ Split based on date: 2022-09-30
  Train: 473 samples (2012-03-31 to 2022-09-30)
  Test:  113 samples (2022-12-31 to 2025-06-30)

✓ Features scaled (RobustScaler)


In [19]:
# ============================================================================
# 6. BALANCED MODELS WITH MODERATE REGULARIZATION
# ============================================================================
print("\n" + "=" * 80)
print("TIME-SERIES CROSS-VALIDATION (5 FOLDS)")
print("Balanced Regularization for Optimal Performance")
print("=" * 80)

tscv = TimeSeriesSplit(n_splits=5)

# BALANCED models - moderate regularization
cv_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.05, random_state=42, max_iter=5000),
    'ElasticNet': ElasticNet(alpha=0.05, l1_ratio=0.7, random_state=42, max_iter=5000),

    'Random Forest': RandomForestRegressor(
        n_estimators=300,
        max_depth=10,  # Balanced (was 12 original, 8 over-regularized)
        min_samples_split=12,  # Balanced
        min_samples_leaf=6,  # Balanced
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),

    'XGBoost': XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,  # Balanced (was 0.03 original, 0.02 over-regularized)
        max_depth=5,  # Balanced
        min_child_weight=3,  # Balanced
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0.1,
        reg_alpha=0.1,  # Light regularization
        reg_lambda=0.5,  # Light regularization
        random_state=42,
        n_jobs=-1,
        early_stopping_rounds=50
    ),

    'LightGBM': LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,  # Balanced (was 25 original, 15 over-regularized)
        min_child_samples=20,  # Balanced
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.5,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),

    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,  # Balanced
        min_samples_split=12,
        min_samples_leaf=6,
        subsample=0.8,
        random_state=42
    )
}

cv_results = {}

print("\nPerforming 5-fold Time-Series Cross-Validation...")
print("-" * 80)

for name, model in cv_models.items():
    print(f"\n{name}:")

    try:
        # For XGBoost with early stopping, we need special handling
        if name == 'XGBoost':
            # Perform CV without early stopping for consistency
            model_cv = XGBRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=5,
                min_child_weight=3,
                subsample=0.8,
                colsample_bytree=0.8,
                gamma=0.1,
                reg_alpha=0.1,
                reg_lambda=0.5,
                random_state=42,
                n_jobs=-1
            )
            cv_scores = cross_validate(
                model_cv,
                X_train_scaled,
                y_train,
                cv=tscv,
                scoring=['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error'],
                n_jobs=-1
            )
        else:
            cv_scores = cross_validate(
                model,
                X_train_scaled,
                y_train,
                cv=tscv,
                scoring=['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error'],
                n_jobs=-1
            )

        cv_scores_rmse = np.sqrt(-cv_scores['test_neg_mean_squared_error'])
        cv_scores_r2 = cv_scores['test_r2']
        cv_scores_mae = -cv_scores['test_neg_mean_absolute_error']

        cv_results[name] = {
            'rmse_mean': cv_scores_rmse.mean(),
            'rmse_std': cv_scores_rmse.std(),
            'r2_mean': cv_scores_r2.mean(),
            'r2_std': cv_scores_r2.std(),
            'mae_mean': cv_scores_mae.mean(),
            'mae_std': cv_scores_mae.std(),
            'rmse_scores': cv_scores_rmse,
            'r2_scores': cv_scores_r2
        }

        print(f"  RMSE: {cv_scores_rmse.mean():.4f} (± {cv_scores_rmse.std():.4f})")
        print(f"  MAE:  {cv_scores_mae.mean():.4f} (± {cv_scores_mae.std():.4f})")
        print(f"  R²:   {cv_scores_r2.mean():.4f} (± {cv_scores_r2.std():.4f})")

    except Exception as e:
        print(f"  ✗ Failed: {str(e)}")


TIME-SERIES CROSS-VALIDATION (5 FOLDS)
Balanced Regularization for Optimal Performance

Performing 5-fold Time-Series Cross-Validation...
--------------------------------------------------------------------------------

Ridge:
  RMSE: 0.9765 (± 0.3893)
  MAE:  0.5896 (± 0.1665)
  R²:   -0.3104 (± 0.8245)

Lasso:
  RMSE: 0.8318 (± 0.3458)
  MAE:  0.5380 (± 0.1724)
  R²:   -0.1024 (± 1.1991)

ElasticNet:
  RMSE: 0.8310 (± 0.3614)
  MAE:  0.5221 (± 0.1806)
  R²:   -0.0892 (± 1.2038)

Random Forest:
  RMSE: 0.6673 (± 0.1893)
  MAE:  0.4954 (± 0.1129)
  R²:   0.4901 (± 0.1680)

XGBoost:
  RMSE: 0.7611 (± 0.1677)
  MAE:  0.5553 (± 0.1013)
  R²:   0.3377 (± 0.1671)

LightGBM:
  RMSE: 0.7831 (± 0.1978)
  MAE:  0.5796 (± 0.1060)
  R²:   0.2666 (± 0.3025)

Gradient Boosting:
  RMSE: 0.7570 (± 0.1303)
  MAE:  0.5496 (± 0.0703)
  R²:   0.3406 (± 0.1109)


In [20]:
# ============================================================================
# 7. CROSS-VALIDATION SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("CROSS-VALIDATION SUMMARY")
print("=" * 80)

cv_summary = pd.DataFrame({
    'Model': list(cv_results.keys()),
    'CV RMSE': [f"{cv_results[m]['rmse_mean']:.4f} ± {cv_results[m]['rmse_std']:.4f}" for m in cv_results.keys()],
    'CV MAE': [f"{cv_results[m]['mae_mean']:.4f} ± {cv_results[m]['mae_std']:.4f}" for m in cv_results.keys()],
    'CV R²': [f"{cv_results[m]['r2_mean']:.4f} ± {cv_results[m]['r2_std']:.4f}" for m in cv_results.keys()],
    'R²_mean': [cv_results[m]['r2_mean'] for m in cv_results.keys()]
})

cv_summary = cv_summary.sort_values('R²_mean', ascending=False)
print("\n" + cv_summary[['Model', 'CV RMSE', 'CV MAE', 'CV R²']].to_string(index=False))

print("\n💡 Performance Target:")
print("  - CV R² > 0.60 = Excellent")
print("  - CV R² 0.50-0.60 = Good")
print("  - CV R² 0.40-0.50 = Acceptable")
print("  - CV R² < 0.40 = Needs improvement")

best_cv_model_name = cv_summary.iloc[0]['Model']
print(f"\n🏆 Best model by cross-validation: {best_cv_model_name}")
print(f"   RMSE: {cv_results[best_cv_model_name]['rmse_mean']:.4f} (± {cv_results[best_cv_model_name]['rmse_std']:.4f})")
print(f"   MAE:  {cv_results[best_cv_model_name]['mae_mean']:.4f} (± {cv_results[best_cv_model_name]['mae_std']:.4f})")
print(f"   R²:   {cv_results[best_cv_model_name]['r2_mean']:.4f} (± {cv_results[best_cv_model_name]['r2_std']:.4f})")


CROSS-VALIDATION SUMMARY

            Model         CV RMSE          CV MAE            CV R²
    Random Forest 0.6673 ± 0.1893 0.4954 ± 0.1129  0.4901 ± 0.1680
Gradient Boosting 0.7570 ± 0.1303 0.5496 ± 0.0703  0.3406 ± 0.1109
          XGBoost 0.7611 ± 0.1677 0.5553 ± 0.1013  0.3377 ± 0.1671
         LightGBM 0.7831 ± 0.1978 0.5796 ± 0.1060  0.2666 ± 0.3025
       ElasticNet 0.8310 ± 0.3614 0.5221 ± 0.1806 -0.0892 ± 1.2038
            Lasso 0.8318 ± 0.3458 0.5380 ± 0.1724 -0.1024 ± 1.1991
            Ridge 0.9765 ± 0.3893 0.5896 ± 0.1665 -0.3104 ± 0.8245

💡 Performance Target:
  - CV R² > 0.60 = Excellent
  - CV R² 0.50-0.60 = Good
  - CV R² 0.40-0.50 = Acceptable
  - CV R² < 0.40 = Needs improvement

🏆 Best model by cross-validation: Random Forest
   RMSE: 0.6673 (± 0.1893)
   MAE:  0.4954 (± 0.1129)
   R²:   0.4901 (± 0.1680)


In [21]:
# ============================================================================
# 8. TRAIN FINAL MODELS AND EVALUATE
# ============================================================================
print("\n" + "=" * 80)
print("FINAL MODEL TRAINING & TEST EVALUATION")
print("=" * 80)

final_results = {}

for name, model in cv_models.items():
    print(f"\n{name}:")
    print("-" * 40)

    try:
        # Special handling for XGBoost with early stopping
        if name == 'XGBoost':
            model.fit(
                X_train_scaled, y_train,
                eval_set=[(X_test_scaled, y_test)],
                verbose=False
            )
        else:
            model.fit(X_train_scaled, y_train)

        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)

        rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
        rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mae_test = mean_absolute_error(y_test, y_pred_test)
        r2_test = r2_score(y_test, y_pred_test)

        overfit_ratio = rmse_test / rmse_train

        final_results[name] = {
            'model': model,
            'predictions': y_pred_test,
            'rmse_train': rmse_train,
            'rmse_test': rmse_test,
            'mae': mae_test,
            'r2': r2_test,
            'overfit_ratio': overfit_ratio
        }

        print(f"  Train RMSE: {rmse_train:.4f}")
        print(f"  Test RMSE:  {rmse_test:.4f}")
        print(f"  Test MAE:   {mae_test:.4f}")
        print(f"  Test R²:    {r2_test:.4f}")
        print(f"  Overfit Ratio: {overfit_ratio:.2f}x {'✓' if overfit_ratio < 1.5 else '⚠'}")

    except Exception as e:
        print(f"  ✗ Failed: {str(e)}")


FINAL MODEL TRAINING & TEST EVALUATION

Ridge:
----------------------------------------
  Train RMSE: 0.5447
  Test RMSE:  0.5225
  Test MAE:   0.3814
  Test R²:    0.7958
  Overfit Ratio: 0.96x ✓

Lasso:
----------------------------------------
  Train RMSE: 0.5760
  Test RMSE:  0.6146
  Test MAE:   0.4499
  Test R²:    0.7174
  Overfit Ratio: 1.07x ✓

ElasticNet:
----------------------------------------
  Train RMSE: 0.5711
  Test RMSE:  0.5931
  Test MAE:   0.4322
  Test R²:    0.7369
  Overfit Ratio: 1.04x ✓

Random Forest:
----------------------------------------
  Train RMSE: 0.4324
  Test RMSE:  0.4851
  Test MAE:   0.3639
  Test R²:    0.8239
  Overfit Ratio: 1.12x ✓

XGBoost:
----------------------------------------
  Train RMSE: 0.2821
  Test RMSE:  0.4870
  Test MAE:   0.3497
  Test R²:    0.8226
  Overfit Ratio: 1.73x ⚠

LightGBM:
----------------------------------------
  Train RMSE: 0.2209
  Test RMSE:  0.5253
  Test MAE:   0.4045
  Test R²:    0.7936
  Overfit Ratio: 2.

In [22]:
# ============================================================================
# 9. COMPREHENSIVE COMPARISON
# ============================================================================
print("\n" + "=" * 80)
print("COMPREHENSIVE MODEL COMPARISON")
print("=" * 80)

comparison_df = pd.DataFrame({
    'Model': list(final_results.keys()),
    'CV R²': [cv_results[m]['r2_mean'] for m in final_results.keys()],
    'CV RMSE': [cv_results[m]['rmse_mean'] for m in final_results.keys()],
    'Test R²': [final_results[m]['r2'] for m in final_results.keys()],
    'Test RMSE': [final_results[m]['rmse_test'] for m in final_results.keys()],
    'Overfit': [final_results[m]['overfit_ratio'] for m in final_results.keys()]
})

comparison_df = comparison_df.sort_values('CV R²', ascending=False)
print("\n" + comparison_df.to_string(index=False))

print("\n📊 What to Look For:")
print("  • High CV R² (>0.60 is excellent)")
print("  • Low Overfit Ratio (<1.5x is good)")
print("  • CV R² close to Test R² (indicates reliability)")


COMPREHENSIVE MODEL COMPARISON

            Model     CV R²  CV RMSE  Test R²  Test RMSE  Overfit
    Random Forest  0.490064 0.667270 0.823943   0.485145 1.121950
Gradient Boosting  0.340601 0.756993 0.822929   0.486540 3.399543
          XGBoost  0.337721 0.761052 0.822624   0.486959 1.726240
         LightGBM  0.266635 0.783052 0.793613   0.525275 2.377614
       ElasticNet -0.089202 0.830986 0.736865   0.593109 1.038510
            Lasso -0.102401 0.831789 0.717441   0.614609 1.067074
            Ridge -0.310411 0.976525 0.795821   0.522456 0.959102

📊 What to Look For:
  • High CV R² (>0.60 is excellent)
  • Low Overfit Ratio (<1.5x is good)
  • CV R² close to Test R² (indicates reliability)


In [23]:
# ============================================================================
# 10. SELECT BEST MODEL
# ============================================================================
print("\n" + "=" * 80)
print("BEST MODEL SELECTION")
print("=" * 80)

best_model_name = comparison_df.iloc[0]['Model']
best_model = final_results[best_model_name]['model']
best_metrics = final_results[best_model_name]

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"\n  Cross-Validation Performance (Most Reliable):")
print(f"    RMSE: {cv_results[best_model_name]['rmse_mean']:.4f} (± {cv_results[best_model_name]['rmse_std']:.4f})")
print(f"    R²:   {cv_results[best_model_name]['r2_mean']:.4f} (± {cv_results[best_model_name]['r2_std']:.4f})")
print(f"\n  Test Set Performance:")
print(f"    RMSE: {best_metrics['rmse_test']:.4f}")
print(f"    R²:   {best_metrics['r2']:.4f}")
print(f"\n  Generalization:")
print(f"    Overfit Ratio: {best_metrics['overfit_ratio']:.2f}x")
print(f"    Status: {'✓ Excellent' if best_metrics['overfit_ratio'] < 1.3 else '✓ Good' if best_metrics['overfit_ratio'] < 1.5 else '⚠ Acceptable'}")

# Performance assessment
cv_r2 = cv_results[best_model_name]['r2_mean']
if cv_r2 >= 0.60:
    assessment = "🎉 EXCELLENT - Production Ready!"
elif cv_r2 >= 0.50:
    assessment = "✅ GOOD - Reliable Performance"
elif cv_r2 >= 0.40:
    assessment = "👍 ACCEPTABLE - Usable"
else:
    assessment = "⚠️ NEEDS IMPROVEMENT"

print(f"\n  Overall Assessment: {assessment}")


BEST MODEL SELECTION

🏆 BEST MODEL: Random Forest

  Cross-Validation Performance (Most Reliable):
    RMSE: 0.6673 (± 0.1893)
    R²:   0.4901 (± 0.1680)

  Test Set Performance:
    RMSE: 0.4851
    R²:   0.8239

  Generalization:
    Overfit Ratio: 1.12x
    Status: ✓ Excellent

  Overall Assessment: 👍 ACCEPTABLE - Usable


In [24]:
# ============================================================================
# 11. FEATURE IMPORTANCE
# ============================================================================
print("\n" + "=" * 80)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': selected_features,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    print(f"\nTop 15 Most Important Features:")
    for idx, row in feature_importance.head(15).iterrows():
        print(f"  {list(feature_importance.head(15).index).index(idx)+1}. {row['Feature']}: {row['Importance']:.4f}")

elif hasattr(best_model, 'coef_'):
    feature_importance = pd.DataFrame({
        'Feature': selected_features,
        'Coefficient': np.abs(best_model.coef_)
    }).sort_values('Coefficient', ascending=False)

    print(f"\nTop 15 Most Important Features:")
    for idx, row in feature_importance.head(15).iterrows():
        print(f"  {list(feature_importance.head(15).index).index(idx)+1}. {row['Feature']}: {row['Coefficient']:.4f}")


FEATURE IMPORTANCE ANALYSIS

Top 15 Most Important Features:
  1. z_score_ma3: 0.1461
  2. z_score_squared: 0.1368
  3. financial_health_score: 0.1284
  4. z_score: 0.1133
  5. z_score_lag1: 0.0748
  6. leverage_risk: 0.0647
  7. mve_to_total_liabilities: 0.0641
  8. z_score_lag3: 0.0464
  9. debt_to_equity_ratio: 0.0391
  10. z_score_lag2: 0.0285
  11. current_ratio: 0.0262
  12. current_ratio_lag1: 0.0207
  13. working_capital_to_total_assets: 0.0180
  14. retained_earnings_to_total_assets: 0.0157
  15. mve_profitability: 0.0141


In [25]:
# ============================================================================
# 12. SAMPLE PREDICTIONS
# ============================================================================
print("\n" + "-" * 80)
print("SAMPLE PREDICTIONS (First 15 Test Samples)")
print("-" * 80)

sample_predictions = pd.DataFrame({
    'Date': test_dates.iloc[:15].dt.date.values,
    'Actual': y_test.iloc[:15].values,
    'Predicted': best_metrics['predictions'][:15],
    'Error': np.abs(y_test.iloc[:15].values - best_metrics['predictions'][:15]),
    'Error %': (np.abs(y_test.iloc[:15].values - best_metrics['predictions'][:15]) /
                (np.abs(y_test.iloc[:15].values) + 0.01) * 100)
})

print("\n" + sample_predictions.to_string(index=False))


--------------------------------------------------------------------------------
SAMPLE PREDICTIONS (First 15 Test Samples)
--------------------------------------------------------------------------------

      Date  Actual  Predicted    Error   Error %
2022-12-31  3.0259   2.722562 0.303338  9.991694
2023-03-31  2.2309   2.763997 0.533097 23.789411
2023-06-30  2.7241   2.773322 0.049222  1.800282
2023-09-30  2.5707   2.605913 0.035213  1.364458
2023-12-31  3.7305   2.867174 0.863326 23.080485
2024-03-31  2.8571   2.689575 0.167525  5.843016
2024-06-30  3.4050   2.982742 0.422258 12.364815
2024-09-30  3.9435   2.880839 1.062661 26.878990
2024-12-31  4.0525   3.153242 0.899258 22.135584
2025-03-31  3.6826   3.055046 0.627554 16.994920
2025-06-30  2.6996   3.212835 0.513235 18.941370
2022-12-31  1.1719   1.133767 0.038133  3.226375
2023-03-31  1.2750   1.109399 0.165601 12.887267
2023-06-30  0.8961   1.173057 0.276957 30.565821
2023-09-30  0.8480   1.037951 0.189951 22.138793


In [26]:
# ============================================================================
# 13. FINAL SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("FINAL RECOMMENDATIONS & INSIGHTS")
print("=" * 80)

print(f"\n✅ RECOMMENDED MODEL: {best_model_name}")
print(f"\n📊 Expected Performance on New Data:")
print(f"   • RMSE: {cv_results[best_model_name]['rmse_mean']:.4f} ± {cv_results[best_model_name]['rmse_std']:.4f}")
print(f"   • R²:   {cv_results[best_model_name]['r2_mean']:.4f} ± {cv_results[best_model_name]['r2_std']:.4f}")
print(f"\n💡 Key Insights:")
print(f"   • Features used: {len(selected_features)} (balanced complexity)")
print(f"   • Overfitting risk: {'Low' if best_metrics['overfit_ratio'] < 1.5 else 'Moderate'}")
print(f"   • Model stability: {'High' if cv_results[best_model_name]['r2_std'] < 0.20 else 'Moderate'}")
print(f"\n⚡ Improvements vs Original:")
print(f"   • Original CV R²: ~0.43 (Random Forest)")
print(f"   • Current CV R²:  {cv_results[best_model_name]['r2_mean']:.4f}")
print(f"   • Improvement: {((cv_results[best_model_name]['r2_mean'] - 0.43) / 0.43 * 100):.1f}%")

print("\n" + "=" * 80)
print("MODEL OPTIMIZATION COMPLETE ✓")
print("=" * 80)


FINAL RECOMMENDATIONS & INSIGHTS

✅ RECOMMENDED MODEL: Random Forest

📊 Expected Performance on New Data:
   • RMSE: 0.6673 ± 0.1893
   • R²:   0.4901 ± 0.1680

💡 Key Insights:
   • Features used: 22 (balanced complexity)
   • Overfitting risk: Low
   • Model stability: High

⚡ Improvements vs Original:
   • Original CV R²: ~0.43 (Random Forest)
   • Current CV R²:  0.4901
   • Improvement: 14.0%

MODEL OPTIMIZATION COMPLETE ✓
